In [7]:
## broken/WIP
## stolen from https://bitbucket.org/deshan/simple-web-crawler
## using as blueprint but will rob-ify

import urllib2
import re 
import settings 
import httplib
import files
import uuid
from urlparse import urlparse
from os.path import join

VALID_MIME_TYPES={'text/html','text/plain','text/xml','text/csv'}
STATUS_CODES={'200','302'}
BASE_SAVE_PATH="./save"
DEFAULT_FILE_FOLDER="files"
DEFAULT_MAIL_FILE="mails.txt"

linkRgx =re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', re.IGNORECASE)
mimeTypeRgx=re.compile(r'^[A-Z,a-z]+[A-Z,a-z,1-9,]*/[A-Z,a-z,1-9]+')

class Crawler: 
    
    fileSavePath=''
    basePath=''
    seed=''
    depth=0

    
    def __init__(self,seed,depth):
        print('############################################################################')
        print('Crawling started.')
        print('Files are saved at path :'+self.basePath)
        parseResult=urlparse(seed)
        self.basePath=join(settings.BASE_SAVE_PATH,parseResult.netloc)
        self.fileSavePath=join(self.basePath,settings.DEFAULT_FILE_FOLDER)
        print( self.fileSavePath )
        self.seed=seed
        self.depth=depth
        
    def getBasePath(self):
        return self.basePath
    
    def getFileSaveth(self):
        return self.fileSavePath
         
    def getResponseHeader(self,url):
        urlParse=urlparse(url)
        conn = httplib.HTTPConnection(urlParse.netloc)
        conn.request("HEAD",urlParse.path)
        res = conn.getresponse()
        return res
    
    def downloadFile(self,url):
        res = self.getResponse(url)
        data=res.read()
        filename = uuid.uuid4()
        files.saveFile(data, str(filename), self.fileSavePath)
        return data
       
    def getResponse(self,url):
        res = urllib2.urlopen(url)
        return res
    
    def harvestLinks(self,text):
        links = set()
        if text is not  None:
            found=linkRgx.findall(text)
            links.update(found)      
        return links
    
    def isValidLink(self,url):
        res=self.getResponseHeader(url)
        if res is not None and str(res.status) in settings.STATUS_CODES:
            head=res.getheader("Content-Type")
            if head is not None:
                mimeType=mimeTypeRgx.match(head).group(0)
                print( '\tContent type:'+mimeType )
                if mimeType  in settings.VALID_MIME_TYPES:
                    return True      
        return False
              
    def start(self):
        self.crawl([self.seed], self.depth)
        
    def crawl(self,url,depth):
        tocrawl=set()
        tocrawl.update(url)
        newLinks=set()
        count=0
        
        print( '==================================================================================')
        print( 'Depth :'+str(depth))
        print( '==================================================================================')
        while tocrawl:
            page=str(tocrawl.pop())
            count=count+1
            print ('['+str(count)+']'+'------------------------------------------')
            try:       
                print( 'Checking Link:'+page+'...' )
                if self.isValidLink(page):
                    print( '\tCrawling...')
                    print( '\tDownloading..' )
                    content=self.downloadFile(page)
                    print( '\tDownloading completed')
                    print( '\tHarvesting links...')
                    foundLinks=self.harvestLinks(content);
                    newLinks.update(foundLinks)
                    print( '\tHarvesting links completed')
                    print(  '\tCrawling completed-'+str(len(foundLinks))+' links found')
                    
                else:
                    print( 'Skipped Error Url:'+page )
                    continue
            except :
                print( 'Error Crawling link:'+page )
            
              
        if(depth>0):
            self.crawl(newLinks, depth=depth-1)  
        else:
            return     


ImportError: No module named 'urllib2'

In [8]:
from  os.path import isdir,join,isfile,exists
from os import listdir,makedirs
import pickle

def getFiles(path):
    onlyFiles=''
    if isdir(path):
        onlyFiles=[]
        onlyFiles = [ join(path,f) for f in listdir(path) if isfile(join(path,f)) ]        
    return onlyFiles
      
def saveFile(data,filename,path):
    if isdir(path):
        savePath=join(path,filename)
        newfile = open(savePath, 'w+')
        newfile.write(data)
        newfile.close()
        
    else:
        createDirectory(path)   
        saveFile(data,filename,path)
        
def createDirectory(directory):
    if not exists(directory):
        makedirs(directory)       
        
def saveListToFile(mailList,filename,path):
    if isdir(path):
        savePath=join(path,filename)
        newfile = open(savePath, 'w+')
        for line in mailList:
            newfile.write(str(line)+'\n')
        newfile.close()
    else:
        createDirectory(path)   
        saveListToFile(mailList,filename,path)        

In [9]:
import crawler
import MailExtracter

# seedUrl = start point url
seedUrl='http://www.google.lk/'

#depth = how deep web page should be crawled
depth=0;

crawl=True
exMails=True

#crawl
if crawl:
    cr= crawler.Crawler(seedUrl,depth)
    cr.start()
     path=cr.getBasePath()
    
#extract mails
 if exMails:
     me=MailExtracter.MailExtracter(path)
     me.start()
     me.printMailList()

IndentationError: unexpected indent (<ipython-input-9-246ab4be1c05>, line 17)

In [1]:
from distutils.core import setup
import unittest
import warnings
warnings.filterwarnings("ignore", "Unknown distribution option")

import sys
# patch distutils if it can't cope with the "classifiers" keyword
if sys.version < '2.2.3':
    from distutils.dist import DistributionMetadata
    DistributionMetadata.classifiers = None
    DistributionMetadata.download_url = None

from BeautifulSoup import __version__

#Make sure all the tests complete.
import BeautifulSoupTests
loader = unittest.TestLoader()
result = unittest.TestResult()
suite = loader.loadTestsFromModule(BeautifulSoupTests)
suite.run(result)
if not result.wasSuccessful():
    print "Unit tests have failed!"
    for l in result.errors, result.failures:
        for case, error in l:
            print "-" * 80
            desc = case.shortDescription()
            if desc:
                print desc
            print error        
    print '''If you see an error like: "'ascii' codec can't encode character...", see\nthe Beautiful Soup documentation:\n http://www.crummy.com/software/BeautifulSoup/documentation.html#Why%20can't%20Beautiful%20Soup%20print%20out%20the%20non-ASCII%20characters%20I%20gave%20it?'''
    print "This might or might not be a problem depending on what you plan to do with\nBeautiful Soup."
    if sys.argv[1] == 'sdist':
        print
        print "I'm not going to make a source distribution since the tests don't pass."
        sys.exit(1)

setup(name="BeautifulSoup",
      version=__version__,
      py_modules=['BeautifulSoup', 'BeautifulSoupTests'],
      description="HTML/XML parser for quick-turnaround applications like screen-scraping.",
      author="Leonard Richardson",
      author_email = "leonardr@segfault.org",
      long_description="""Beautiful Soup parses arbitrarily invalid SGML and provides a variety of methods and Pythonic idioms for iterating and searching the parse tree.""",
      classifiers=["Development Status :: 5 - Production/Stable",
                   "Intended Audience :: Developers",
                   "License :: OSI Approved :: Python Software Foundation License",
                   "Programming Language :: Python",
                   "Topic :: Text Processing :: Markup :: HTML",
                   "Topic :: Text Processing :: Markup :: XML",
                   "Topic :: Text Processing :: Markup :: SGML",
                   "Topic :: Software Development :: Libraries :: Python Modules",
                   ],
      url="http://www.crummy.com/software/BeautifulSoup/",
      license="BSD",
      download_url="http://www.crummy.com/software/BeautifulSoup/download/"
      )
    
    # Send announce to:
    #   python-announce@python.org
    #   python-list@python.org

SyntaxError: Missing parentheses in call to 'print' (<ipython-input-1-57dac3e73c30>, line 22)

## In pseudocode

We do unordered BFS to crawl.

- We initialize a hashtable of to_visit_table; our queue.
- We initialize a hashtable of pages visited_table.
- We insert a first page into the queue.
- We select the next page in the queue, we first remove it from the hashtable, and insert it into the pages visited table.
- We visit the page, and store the page however we want to.
- Scan the page for links.
- For all links in the page, we check the pages visited, if not in visited_table, we insert it into the to_visit_table.
- Then we return to the queue for the next page to visit.

We can parallelize the crawl by using agents to process the queue.

The actors will have states.

- visiting_page
- waiting

Our parent process will end when all of the child actors are in the waiting state.